In [11]:
import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from dataset import EEGDataset
from torch.utils.data import random_split


In [4]:
# load in the dataset

#raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'
raw_data_dir = '../data'

trainingNights = 9
testNights = 2

print("Test set")
ds2 = EEGDataset(raw_data_dir,testNights, 250, skips = trainingNights)

print("Training set")
ds1 = EEGDataset(raw_data_dir,trainingNights, 250, skips = 0) #Instantiate a dataset using the directory of data, amount of night to include and amount of samples in a segment



Test set
../data/study_1A_mat_simple/S_01/night_1/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_1/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_2/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_2/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_3/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_3/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_4/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_4/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_1/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_1/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_2/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_2/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_3/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_3/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_4/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_4/EEG_raw_2

In [5]:
#Calculate class imbalance
balancing_dataloader = DataLoader(ds1, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
artefacts = 0
good_samples = 0
for batch, (X, y) in enumerate(balancing_dataloader):
    if y == 1:
        artefacts += 1
    else:
        good_samples += 1
    if batch > 100000:
        break

class_ratio = good_samples/artefacts
print("debug")


debug


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.BatchNorm1d(250),
            nn.Linear(250, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid(), 
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


In [7]:
learning_rate = 1e-3
batch_size = 64


model = NeuralNetwork()
model.to(device)

#pos_weight: amount of positive examples compared to negative examples. Calculate as: negative_examples/positive_examples
loss = nn.BCEWithLogitsLoss(pos_weight = class_ratio*torch.ones([batch_size]).to(device)) 

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)# Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
        pred = pred.reshape(-1)
        pred = pred.to(device)
        yFloat = y.type(torch.FloatTensor).to(device)
        
        loss = loss_fn(pred, yFloat)
        


        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            #print("Predicted values:")
            #print(pred)
            #print("Actual values:")
            #print(yFloat)



def test_loop(dataloader_test, model, loss_fn, test_set = True):
    size = len(dataloader_test.dataset)
    num_batches = len(dataloader_test)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader_test:
            X = X.to(device)
            y = y.to(device)
            pred = model(X).reshape(-1).to(device) # Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
            test_loss += loss_fn(pred, y.type(torch.FloatTensor).to(device)).item()
            correct += (pred.round() == y).type(torch.float).sum().item()
            

    test_loss /= num_batches
    correct /= size
    if test_set:
        print(f"Test set Error: \n Test Set Accuracy: {(100*correct):>0.5f}%, Avg Test Set loss: {test_loss:>8f} \n")
    else:
        print(f"Training Set Error: \n Training Set Accuracy: {(100*correct):>0.5f}%, Avg Training Set loss: {test_loss:>8f} \n")


In [9]:
# Split data into train and test data (Commented out: Changed to loading different dataset class instances)
#trainSamples = int(ds1.__len__()*0.7)
#testSamples = int(ds1.__len__() - trainSamples)
#training_data, test_data = random_split(ds1, (trainSamples,testSamples), generator=torch.Generator().manual_seed(42))

#train_dataloader = DataLoader(training_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
#test_dataloader = DataLoader(test_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss


train_dataloader = DataLoader(ds1, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
test_dataloader = DataLoader(ds2, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss



In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train_loop(train_dataloader, model, loss, optimizer)    
    test_loop(test_dataloader, model, loss)
    test_loop(train_dataloader, model, loss, test_set = False)

print("Done!")

Epoch 1
-------------------------------
loss: 0.719505  [    0/6508495]
loss: 0.708488  [64000/6508495]
loss: 0.712196  [128000/6508495]
loss: 0.680704  [192000/6508495]
loss: 0.656429  [256000/6508495]
loss: 0.628320  [320000/6508495]
loss: 0.595436  [384000/6508495]
loss: 0.578245  [448000/6508495]
loss: 0.572842  [512000/6508495]
loss: 0.617983  [576000/6508495]
loss: 0.583535  [640000/6508495]
loss: 0.584873  [704000/6508495]
loss: 0.585472  [768000/6508495]
loss: 0.608247  [832000/6508495]
loss: 0.572189  [896000/6508495]
loss: 0.579306  [960000/6508495]
loss: 0.586438  [1024000/6508495]
loss: 0.590747  [1088000/6508495]
loss: 0.584993  [1152000/6508495]
loss: 0.563189  [1216000/6508495]
loss: 0.581698  [1280000/6508495]
loss: 0.587315  [1344000/6508495]
loss: 0.579288  [1408000/6508495]
loss: 0.582122  [1472000/6508495]
loss: 0.582383  [1536000/6508495]
loss: 0.572791  [1600000/6508495]
loss: 0.594168  [1664000/6508495]
loss: 0.637254  [1728000/6508495]
loss: 0.633589  [1792000/6

KeyboardInterrupt: 